In [0]:
# 구글 드라이브 마운트, 주피터에선 사용 X
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# 필수 라이브러리 import
%tensorflow_version 1.x
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Embedding, Dense, LSTM, GRU
import pickle
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13362943725948410153
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16347638019605862103
physical_device_desc: "device: XLA_CPU device"
]


In [0]:
# 변수 설정
normal_path = '/content/gdrive/My Drive/캡스톤/ben/' #정상파일 경로
mal_path = '/content/gdrive/My Drive/캡스톤/mnemonic/' #악성파일 경로
model_path = '/content/gdrive/My Drive/캡스톤/' #모델 가중치 저장 경로
max_len = 32 #n-그램 설정
batch_size = 1024
epochs = 10
vocab_size = 1001
count_dict = {}

In [0]:
### word_count 생성 ###
def word_count(block):
  for line in block:
      for mnemonics in line:
        for mnemonic in mnemonics:
          if mnemonic not in count_dict:
            count_dict[mnemonic] = 1
          else:
            count_dict[mnemonic] += 1

# 데이터 불러오기
def data_load_for_word_count(path):
  file_names = os.listdir(path)
  file_names = file_names[0:500]
  data = []
  for index, file_name in enumerate(file_names):
    print("\r{} / {}".format(index + 1, len(file_names)), end="")
    f = open(path + file_name, 'rb')
    mnemonic = pickle.load(f)
    word_count(mnemonic)
    #yield mnemonic
    f.close()
    
  print()

In [0]:
def gen_word_index():
  word_index = {}
  word_index_dec = sorted(count_dict, key=lambda k : count_dict[k], reverse=True)
  for i in range(len(word_index_dec)):
      if i < (vocab_size-1):
        word_index[word_index_dec[i]] = i + 1
  return word_index

In [0]:
def get_index(word, word_index):
  if word in word_index:
    return [word_index[word]]
  else:
    return [0]

def seqs_gen(path, word_index, type=None):
  file_names = os.listdir(path)
  if type == 'train':
    file_names = file_names[:400]
  elif type == 'valid':
    file_names = file_names[400:500]
  zero_vec = [0] * (max_len - 1)
  for index, file_name in enumerate(file_names):
    f = open(path + file_name, 'rb')
    block = pickle.load(f)
    for line in block:
      for mnemonics in line:
        if len(mnemonics) == 0:
          seqs = zero_vec + [0]
          x = np.array([seqs[:-1]])
          y = to_categorical([seqs[-1]], num_classes=vocab_size)
          yield (x, y)
        else:
          seqs = zero_vec + get_index(mnemonics[0], word_index)
          x = np.array([seqs[:-1]])
          y = to_categorical([seqs[-1]], num_classes=vocab_size)
          yield (x, y)
          for i in range(1, len(mnemonics)):
            seqs = seqs[1:] + get_index(mnemonics[i], word_index)
            x = np.array([seqs[:-1]])
            y = to_categorical([seqs[-1]], num_classes=vocab_size)
            yield (x, y)
    f.close()

def size_get(path, type=None):
  file_names = os.listdir(path)
  if type == 'train':
    file_names = file_names[:400]
  elif type == 'valid':
    file_names = file_names[400:500]
  
  len_files = []
  for index, file_name in enumerate(file_names):
    f = open(path + file_name, 'rb')
    file = pickle.load(f)
    file_size = 0
    for block in file:
      block_size = 0
      for line in block:
        block_size += len(line)
      file_size += block_size

    len_files.append(file_size)
  
  return len_files

In [0]:
f = open(model_path + "count_dict_400.dat", 'rb')
count_dict = pickle.load(f)
f.close()
print(count_dict)

{'mov': 39335710, 'test': 4072773, 'push': 16837173, 'jnz': 2929016, 'xor': 2934410, 'div': 26603, 'cmp': 5624927, 'ja': 255962, 'jbe': 345479, 'lea': 5610665, 'imul': 316683, 'call': 10288440, 'add': 4173930, 'pop': 5260750, 'retn': 2163397, 'inc': 770498, 'jb': 434538, 'sub': 1723280, 'shr': 456384, 'stosd': 60137, 'and': 1752935, 'stosb': 43916, 'movsd': 541711, 'movsb': 4839, 'jz': 4333705, 'jnb': 252121, 'jmp': 3645206, 'dec': 294485, 'movzx': 763026, 'jle': 307910, 'neg': 150400, 'sbb': 138804, 'or': 869239, 'cmpsb': 7070, 'jge': 152645, 'setnz': 74570, 'movsx': 122091, 'jl': 429191, 'shl': 428986, 'jg': 330646, 'nop': 196722, 'adc': 336071, 'js': 251864, 'jns': 121666, 'setz': 69031, 'sar': 276605, 'cdq': 156063, 'mul': 42219, 'setnle': 23240, 'setle': 2061, 'fild': 33303, 'fadd': 48050, 'fld': 463523, 'fstp': 374349, 'fst': 53635, 'fxch': 96035, 'fucompp': 5499, 'fnstsw': 47825, 'jnp': 19680, 'setnbe': 3407, 'fcomp': 18439, 'jp': 27641, 'fchs': 10942, 'setb': 4102, 'setnl': 364

In [0]:
word_index = gen_word_index()

In [0]:
train_lens = size_get(normal_path, 'train')
train_size = sum(train_lens)

In [0]:
# 모델 정의
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=max_len-1))
model.add(LSTM(128))
model.add(Dense(vocab_size, activation='softmax'))

In [0]:
# 모델 최초 생성
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.fit(seqs_gen(normal_path, word_index, 'train'),
          steps_per_epoch=train_size/8192,
          epochs=epochs,
          verbose=2)

 - 225s - loss: 1.6780 - accuracy: 0.4852
Epoch 5/10
 - 225s - loss: 1.6785 - accuracy: 0.5086
Epoch 6/10
 - 221s - loss: 1.7671 - accuracy: 0.4964
Epoch 7/10
 - 227s - loss: 2.1876 - accuracy: 0.3625
Epoch 8/10
 - 230s - loss: 1.8051 - accuracy: 0.4370
Epoch 9/10
 - 223s - loss: 1.9232 - accuracy: 0.4326
Epoch 10/10
 - 219s - loss: 1.8118 - accuracy: 0.4432


In [0]:
model.save_weights(model_path + "lstm_model.h5")

In [0]:
# 모델 가중치 불러오기
model.load_weights(model_path + 'lstm_model.h5')
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
# 모델 평가 함수
def evaluate(generator, gen_size, dataset):
  threshold = round(vocab_size * 0.9)
  count = 0
  well = 0
  index = 0
  abnormal = 0
  abnormals = []
  
  for i in range(gen_size):
    if i % 1000 == 0:
      print("\r{} / {}".format(i + 1, gen_size), end="")
    
    x, y = next(generator)
    pred_y = model.predict(x)[0]

    valid_index = np.where(y == 1)[0]

    if valid_index == 0:
      pred_per = 0
    else:
      pred_per = pred_y[valid_index]
      pred_y.sort()

    if pred_per >= pred_y[threshold]:
      well += 1

    count += 1
    if count == dataset[index]:
      abnormal = count - well
      score = abnormal / dataset[index]
      index += 1
      count = 0
      well = 0
      abnormals.append(score)
      f = open(model_path + "valid_evaluate.txt", 'a')
      f.write(str(score))
      f.close()
  print()
  abnormals = np.array(abnormals)
  return abnormals.mean()

In [0]:
# 모델 평가
#x_valid, y_valid = list()
valid_lens = size_get(normal_path, 'valid')
valid_size = sum(valid_lens)

In [0]:
import time

start_time = time.time()
valid_abnormal = evaluate(seqs_gen(normal_path, word_index, 'valid'), valid_size, valid_lens)
f = open(model_path + "valid_evaluate.txt", 'a')
f.write(str(valid_abnormal))
f.close()
end_time = time.time() - start_time
print(end_time)

NameError: ignored